**Importing Libraries and Data**

In [ ]:
#Importing libraries
import pandas as pd
import statsmodels.api as sm
from sklearn.metrics import root_mean_squared_error
from statsmodels.tsa.stattools import acf
import matplotlib.pyplot as plt
import prophet
import numpy as np
import warnings
import gc
import os
import sys

#Ignoring warnings
warnings.filterwarnings("ignore")

#Reading in data and extracting unique account values
BoostedTransactions = pd.read_csv("Longer Subset of Transaction Data (14 Months).csv")
AccountIDs = BoostedTransactions["AccountId"].values
UniqueAccounts = set(AccountIDs)

#Converting transaction dates into months
BoostedTransactions["TransactionDate"] = pd.to_datetime(BoostedTransactions["TransactionDate"])
BoostedTransactions["TransactionDate"] = BoostedTransactions["TransactionDate"].dt.to_period("M").dt.to_timestamp()

#Grouping transactions by account and month and summating each grouping's net transaction amount
GroupedSumOfTransactionsByAccountAndMonth = BoostedTransactions.groupby(by=["AccountId", "TransactionDate"]).agg(["sum"])["Amount"]
print(GroupedSumOfTransactionsByAccountAndMonth)

#Creating a list to store all accounts' time series
DataFrames = []

#Imputing 0 values into the gaps in each account's time series
for i in range(0, len(UniqueAccounts)):
  ResetDataFrame = GroupedSumOfTransactionsByAccountAndMonth.reset_index()
  TimeSeries = ResetDataFrame[ResetDataFrame["AccountId"] == list(UniqueAccounts)[i]]
  TimeSeries = TimeSeries.set_index("TransactionDate")

  TimeSeries = TimeSeries.resample("MS").sum()
  for j in range(0, len(TimeSeries)):
    if TimeSeries.iloc[j]["AccountId"] == 0:
      TimeSeries.at[TimeSeries.index[j], "AccountId"] = list(UniqueAccounts)[i]

  DataFrames.append(TimeSeries)

#Combining all accounts' time series into a new grouped dataset
NewGroupedDataset = pd.concat(DataFrames)
NewGroupedDataset = NewGroupedDataset.reset_index()
NewGroupedDataset = NewGroupedDataset.set_index(["AccountId", "TransactionDate"])
print(NewGroupedDataset)

#Calculating the cumulative transaction sum over the time period for each account
GroupedCumulativeSumOfTransactionsByAccountAndMonth = NewGroupedDataset.groupby(level=0).cumsum().reset_index()
print(GroupedCumulativeSumOfTransactionsByAccountAndMonth)

                                                          sum
AccountId                            TransactionDate         
0003a5ae-0c77-4372-b44d-882ef9874a28 2019-05-01      -5541.31
                                     2019-06-01      -3601.46
                                     2019-07-01       3151.74
                                     2019-08-01      -4801.13
                                     2019-09-01       3890.05
...                                                       ...
fff7f00c-c869-4310-b705-4503538f5ecf 2020-03-01       2974.84
                                     2020-04-01        382.93
                                     2020-05-01        166.15
                                     2020-06-01       -550.89
                                     2020-07-01      -2721.18

[7632 rows x 1 columns]
                                                           sum
AccountId                            TransactionDate          
49f91fce-526b-46fa-9ac0-8c80b6eae3c7 2019-0

**Key Prophet Loop**

In [ ]:
#Creating models and forecasting for all accounts
for i in UniqueAccounts:
    print(i)
    try:
        #Selecting only the selected account's time series
        ForecastingTimeSeries = GroupedCumulativeSumOfTransactionsByAccountAndMonth[GroupedCumulativeSumOfTransactionsByAccountAndMonth["AccountId"] == i][["TransactionDate", "sum"]]

        #Calculating the cutoff between training and testing sets
        TrainEnd = int(0.7 * len(ForecastingTimeSeries))

        #Splitting the data into training and testing sets and adjusting column names in each
        TrainData = ForecastingTimeSeries[:TrainEnd]
        TestData = ForecastingTimeSeries[TrainEnd:]

        TrainData.columns = ["ds", "y"]
        TestData.columns = ["ds", "y"]

        #Declaring and training the model
        Model = prophet.Prophet()
        Model.fit(TrainData)

        #Performing a test forecast
        Forecast = Model.predict(TestData)

        #Calculating error rates and appending them into the errors CSV
        try:
          RMSE = root_mean_squared_error(Forecast["yhat"], TestData["y"])
          SI = abs(RMSE/abs(TestData["y"]).mean())
          AccountError = pd.DataFrame({"AccountID" : [i],
                                        "RMSE" : [RMSE],
                                        "SI" : [SI]})
          AccountError.to_csv("Prophet Longer Subset Errors (Cumulative).csv", mode='a', header=not os.path.exists("Prophet Longer Subset Errors (Cumulative).csv"), index=False)
        #Where a ValueError occurs, the loop is broken
        except ValueError:
          print("ValueError encountered")
          break

        #Forecasting the next 12 months
        Future = Model.make_future_dataframe(periods=12, freq="MS")
        FutureForecast = Model.predict(Future)

        FutureForecastTail = FutureForecast.tail(12)
        FutureForecastValues = FutureForecastTail["yhat"].values

        #Calculating starting amount, ending amount and net change and appending them into the forecasts CSV
        NetChange = FutureForecastValues[-1] - FutureForecastValues[0]
        StartingAmount = FutureForecastValues[0]
        EndingAmount = FutureForecastValues[-1]

        ForecastData = pd.DataFrame({"AccountID" : [i],
                                     "Net Change over Forecast Period" : [NetChange],
                                     "Starting Amount" : [StartingAmount],
                                     "Ending Amount" : [EndingAmount]})

        ForecastData.to_csv("Prophet Longer Subset Forecasts (Cumulative).csv", mode='a', header=not os.path.exists("Prophet Longer Subset Forecasts (Cumulative).csv"), index=False)

        print("Processed account number",i)
    #Where an error occurs, the user is informed about it
    except Exception as e:
        print(f"Error encountered processing account number {i}.")
        ErrorType, ErrorObject, ErrorTraceback = sys.exc_info()

        ErrorFilename = os.path.split(
            ErrorTraceback.tb_frame.f_code.co_filename
        )[1]

        ErrorMessage = str(e)

        ErrorLineNumber = ErrorTraceback.tb_lineno

        print(f'Exception Type: {ErrorType}')

        print(f'Exception Filename: {ErrorFilename}')

        print(f'Exception Line Number: {ErrorLineNumber}')

        print(f'Exception Message: {ErrorMessage}')
        break
    #Deleting model data to conserve RAM
    finally:
        del Model, Forecast, TrainData, TestData, ForecastingTimeSeries
        gc.collect()


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/q9frsemt.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/obp3oiay.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=4446', 'data', 'file=/tmp/tmptpy2zrvj/q9frsemt.json', 'init=/tmp/tmptpy2zrvj/obp3oiay.json', 'output', 'file=/tmp/tmptpy2zrvj/prophet_models6nehh21/prophet_model-20250807113426.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:34:26 - cmdstanpy - INFO - Chain [1] start processing
INF

49f91fce-526b-46fa-9ac0-8c80b6eae3c7
Processed account number 49f91fce-526b-46fa-9ac0-8c80b6eae3c7
c4886893-3a39-417d-acc8-80810fca1e7e


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/162n6ky2.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/_rz5yxu3.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=12709', 'data', 'file=/tmp/tmptpy2zrvj/162n6ky2.json', 'init=/tmp/tmptpy2zrvj/_rz5yxu3.json', 'output', 'file=/tmp/tmptpy2zrvj/prophet_modele8sdz5wg/prophet_model-20250807113426.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:34:26 - cmdstanpy - INFO - Chain [1] start processing
IN

Processed account number c4886893-3a39-417d-acc8-80810fca1e7e
5ffc25d1-97f4-4eb2-b31e-8e1a5cb39aa8
Processed account number 5ffc25d1-97f4-4eb2-b31e-8e1a5cb39aa8
7e8334c1-0da0-4993-a413-ea18202daae9


11:34:26 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/d4q_yozz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/rfnz9yq2.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20399', 'data', 'file=/tmp/tmptpy2zrvj/d4q_yozz.json', 'init=/tmp/tmptpy2zrvj/rfnz9yq2.json', 'output', 'file=/tmp/tmptpy2zrvj/prophet_modelt5p35sgw/prophet_model-20250807113426.csv', 'method=optimiz

...

16c0e231-2dd6-42f6-908a-a668c98c9c21
Processed account number 16c0e231-2dd6-42f6-908a-a668c98c9c21


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/2j5nxcqr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/a3cnthhq.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=93701', 'data', 'file=/tmp/tmptpy2zrvj/2j5nxcqr.json', 'init=/tmp/tmptpy2zrvj/a3cnthhq.json', 'output', 'file=/tmp/tmptpy2zrvj/prophet_model96dll25x/prophet_model-20250807113614.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:36:14 - cmdstanpy - INFO - Chain [1] start processing
IN

b0926892-ec8b-4ffd-b86e-d909fff85313
Processed account number b0926892-ec8b-4ffd-b86e-d909fff85313


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:n_changepoints greater than number of observations. Using 7.
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/mxeh9_y9.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmptpy2zrvj/skyovpb8.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=1372', 'data', 'file=/tmp/tmptpy2zrvj/mxeh9_y9.json', 'init=/tmp/tmptpy2zrvj/skyovpb8.json', 'output', 'file=/tmp/tmptpy2zrvj/prophet_modelz4syg9e4/prophet_model-20250807113614.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
11:36:14 - cmdstanpy - INFO - Chain [1] start processing
INF

8432e6bc-235d-4ac2-b1c6-18fc20f82f43
Processed account number 8432e6bc-235d-4ac2-b1c6-18fc20f82f43


**Calculating Error Statistics**

In [ ]:
#Calculating averaged error statistics
Errors = pd.read_csv("Prophet Longer Subset Errors (Cumulative).csv")
MeanRMSE = Errors["RMSE"].mean()
print("Mean RMSE:",MeanRMSE)
MedianRMSE = Errors["RMSE"].median()
print("Median RMSE:",MedianRMSE)
MeanSI = Errors["SI"].mean()
print("Mean SI:",MeanSI)
MedianSI = Errors["SI"].median()
print("Median SI:",MedianSI)

Mean RMSE: 175260.72476009012
Median RMSE: 14466.594670272016
Mean SI: 0.4442843331090964
Median SI: 0.2792133251437129
